## Learning Objectives

- How we can dockerize the ML or DL model

- How we can deploy the docker image on AWS

- Learn about useful docker commands

## What is Docker? 

- Docker is a container management service

- The keywords of Docker are develop, ship and run anywhere

-  It provides tools for simplifying DevOps by enabling developers to create templates called images that can be used to create lightweight virtual machines called containers, which include their applications and all of their applications’ dependencies.

## How we can deploy the ML or DL model on cloud (AWS)?

### First step for Dockerize the Flask API

In [ ]:
# Train, Evaluate and Save the DL Model


from __future__ import print_function
import keras
from keras.datasets import mnist
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten
from keras.layers import Conv2D, MaxPooling2D
from keras import backend as K
import numpy as np


batch_size = 128
num_classes = 10
epochs = 1

# input image dimensions
img_rows, img_cols = 28, 28

# the data, split between train and test sets
(x_train, y_train), (x_test, y_test) = mnist.load_data()

if K.image_data_format() == 'channels_first':
    x_train = x_train.reshape(x_train.shape[0], 1, img_rows, img_cols)
    x_test = x_test.reshape(x_test.shape[0], 1, img_rows, img_cols)
    input_shape = (1, img_rows, img_cols)
else:
    x_train = x_train.reshape(x_train.shape[0], img_rows, img_cols, 1)
    x_test = x_test.reshape(x_test.shape[0], img_rows, img_cols, 1)
    input_shape = (img_rows, img_cols, 1)

print('input_shape')
print(input_shape)
x_train = x_train.astype('float32')
x_test = x_test.astype('float32')
x_train /= 255
x_test /= 255
print('x_train shape:', x_train.shape)
print(x_train.shape[0], 'train samples')
print(x_test.shape[0], 'test samples')

# convert class vectors to binary class matrices
y_train = keras.utils.to_categorical(y_train, num_classes)
y_test = keras.utils.to_categorical(y_test, num_classes)

model = Sequential()
model.add(Conv2D(32, kernel_size=(3, 3),
                 activation='relu',
                 input_shape=input_shape))
model.add(Conv2D(64, (3, 3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))
model.add(Flatten())
model.add(Dense(128, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(num_classes, activation='softmax'))

model.compile(loss=keras.losses.categorical_crossentropy,
              optimizer=keras.optimizers.Adadelta(),
              metrics=['accuracy'])

model.fit(x_train, y_train,
          batch_size=batch_size,
          epochs=epochs,
          verbose=1,
          validation_data=(x_test, y_test))
# model.save('my_model.h5')

# Save the weights
model.save_weights('model_weights.h5')
# Save the model architecture
with open('model_architecture.json', 'w') as f:
    f.write(model.to_json())

score = model.evaluate(x_test, y_test, verbose=0)
print('Test loss:', score[0])
print('Test accuracy:', score[1])
print('M:')
print(y_test[0])
print(x_test[0].shape)
x = x_test[0].reshape(1, 28, 28, 1)
out = model.predict(x)
print(out[0])
print(np.argmax(out[0]))

### Second step for Dockerize the Flask API

In [ ]:
# Make a flask API for our DL Model


from keras.preprocessing.image import img_to_array
from keras.models import load_model
from flask_restplus import Api, Resource, fields
from flask import Flask, request, jsonify
import numpy as np
from werkzeug.datastructures import FileStorage
from PIL import Image
from keras.models import model_from_json
import tensorflow as tf


app = Flask(__name__)
api = Api(app, version='1.0', title='MNIST Classification', description='CNN for Mnist')
ns = api.namespace('Make_School', description='Methods')

single_parser = api.parser()
single_parser.add_argument('file', location='files',
                           type=FileStorage, required=True)

model = load_model('my_model.h5')
graph = tf.get_default_graph()

# Model reconstruction from JSON file
# with open('model_architecture.json', 'r') as f:
#     model = model_from_json(f.read())
#
# # Load weights into the new model
# model.load_weights('model_weights.h5')


@ns.route('/prediction')
class CNNPrediction(Resource):
    """Uploads your data to the CNN"""
    @api.doc(parser=single_parser, description='Upload an mnist image')
    def post(self):
        args = single_parser.parse_args()
        image_file = args.file
        image_file.save('milad.png')
        img = Image.open('milad.png')
        image_red = img.resize((28, 28))
        image = img_to_array(image_red)
        print(image.shape)
        x = image.reshape(1, 28, 28, 1)
        x = x/255
        # This is not good, because this code implies that the model will be
        # loaded each and every time a new request comes in.
        # model = load_model('my_model.h5')
        with graph.as_default():
            out = model.predict(x)
        print(out[0])
        print(np.argmax(out[0]))
        r = np.argmax(out[0])

        return {'prediction': str(r)}


if __name__ == '__main__':
    app.run(host='0.0.0.0', port=8000)

### Third step for Dockerize the Flask API

- Make Dockerfile
    - Allows to list a succession of commands describing how to build a container
    

- All the packges needed in `requirements.txt`

In [ ]:
# Dockerfile 
FROM python:3.6
COPY . /app
WORKDIR /app
RUN pip install -r requirements.txt
ENTRYPOINT ["python"]
CMD ["flask_example_1.py"]

In [ ]:
flask
flask_restplus
tensorflow
keras
numpy
Pillow
werkzeug

### Fourth step for Dockerize the Flask API

In [ ]:
docker build -t flask_keras_docker:latest .
    
docker run -d -p 8000:8000 flask_keras_docker

#### How to push to DockerHub


In [ ]:
To push:
docker login
docker tag local-image:tagname reponame:tagname
docker push reponame:tagname (docker push 88696316/flask_keras:latest)

#### How to pull from DockerHub

In [ ]:
docker pull 88696316/flask_keras
docker run -p 8000:8000 88696316/flask_keras

## How we can deploy the ML or DL model on cloud (AWS)?

We use Amazon Elastic Containers Service (ECS)

Lets watch: https://www.youtube.com/watch?v=-Vsuzi4OByY

1- `login_command=$(aws ecr get-login --no-include-email --region us-east-1)`

2- `docker tag flask_keras_docker:latest 312741836182.dkr.ecr.us-east-1.amazonaws.com/flask-keras-on-fargate-aws:latest`

3- `docker push 312741836182.dkr.ecr.us-east-1.amazonaws.com/flask-keras-on-fargate-aws:latest`

#### Edit Inbound Rule

### Useful Docker Commands:

In [ ]:
docker ps -a, docker ps, docker images
docker logs <container_id>
docker stop <container_id>, docker kill <container_id>